In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# /content/drive/My Drive/ = 구글 드라이브(내 드라이브) 경로

!ls -l '/content/drive/My Drive/Colab Notebooks/datasets/creditCardFraud.zip'

-rw------- 1 root root 69155672 Mar  9 02:39 '/content/drive/My Drive/Colab Notebooks/datasets/creditCardFraud.zip'


In [ ]:
!unzip /content/drive/My\ Drive/Colab\ Notebooks/datasets/creditCardFraud.zip

Archive:  /content/drive/My Drive/Colab Notebooks/datasets/creditCardFraud.zip
  inflating: creditcard.csv          


In [ ]:
!ls -l

total 147304
-rw-r--r-- 1 root root 150828752 Sep 20  2019 creditcard.csv
drwx------ 5 root root      4096 Mar  9 08:42 drive
drwxr-xr-x 1 root root      4096 Mar  5 14:37 sample_data


# 1, Data Preprocessing

In [68]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score
from lightgbm import LGBMClassifier
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier

In [ ]:
%%time
import pandas as pd

DF = pd.read_csv('creditcard.csv')
DF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [ ]:
DF.drop('Time', axis=1, inplace=True)
DF.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


* 암호화된 데이터는 이해도가 떨어질 수 밖에 없다.
* 따라서 어느정도 상관관계가 있는지 알 수 없으며, 비즈니스 입장에서 무엇을 고려해야할 지도 알 수 없다.
* 어느정도 오차를 감안해야할 지도 알 수 없다
* 따라서 **오차율을 낮추는 방식**으로 접근할 수 밖에.

In [47]:
X = DF.iloc[:,:-1]
y = DF['Class']

In [48]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, 
                                                    stratify = y, 
                                                    random_state = 2045)

print('Train Data : ', X_train.shape, y_train.shape)
print('Test Data : ', X_test.shape, y_test.shape)

Train Data :  (199364, 29) (199364,)
Test Data :  (85443, 29) (85443,)


In [ ]:
# accuracy_score/ precision_score/ recall_score
Model_Result = []

# 2, Logistic Regression

## 1) 모델 생성

In [65]:
from sklearn.metrics import accuracy_score, recall_score, precision_score
%%time
LR_result = []
count = 1
a = [0.1, 0.3, 0.5, 0.7]

for n in a:
  LR = LogisticRegression(C = n, penalty='l2', multi_class='multinomial', n_jobs=-1)
  LR.fit(X_train, y_train)

  accuracy_score= accuracy_score(y_test,LR.predict(X_test))
  precision_score = precision_score(y_test,LR.predict(X_test))
  recall_score = recall_score(y_test,LR.predict(X_test))

  LR_result.append([accuracy_score,precision_score, recall_score])
  print(f'num : {count}, accuracy_score : {accuracy_score}, precision_score : {precision_score}, recall_score : {recall_score}'
  count += 1

TypeError: ignored

In [ ]:
LR_result

# 3, Decision Tree Classification

In [82]:
%%time
from sklearn.metrics import accuracy_score, recall_score, precision_score
param_max_features = [2,3,5,7]
param_max_depth = [2,5,7,9]
DT_result = pd.DataFrame(data=None, columns=('Num','max_features','max_depth','accuracy_score','precision_score','recall_score'))
Num = 1


for mf in param_max_features:
  for md in param_max_depth:

    # 모델 학습/예측
    DT = DecisionTreeClassifier(max_features = mf, max_depth = md , criterion = 'entropy' , random_state = 2045)
    DT.fit(X_train  , y_train)

    y_hat = DT.predict(X_test)
    ac= accuracy_score(y_test,y_hat)
    pr = precision_score(y_test,y_hat)
    re = recall_score(y_test,y_hat)

    res =  {'Num':Num, 'max_features':mf, 'max_depth':md, 'accuracy_score':ac, 'precision_score':pr, 'recall_score':re}
    DT_result.append(res, ignore_index=True)
    #print(f" Num : {Num} , max_features : {mf}  , max_depth : {md}, accuracy_score : {ac},precision_score : {pr},recall_score : {re}")
    print(res)
    Num+=1
  

 Num : 1 , max_features : 2  , max_depth : 2, accuracy_score : 0.9991807403766253,precision_score : 0.8362068965517241,recall_score : 0.6554054054054054
 Num : 2 , max_features : 2  , max_depth : 5, accuracy_score : 0.9993328885923949,precision_score : 0.8760330578512396,recall_score : 0.7162162162162162
 Num : 3 , max_features : 2  , max_depth : 7, accuracy_score : 0.9993562960102056,precision_score : 0.8345323741007195,recall_score : 0.7837837837837838
 Num : 4 , max_features : 2  , max_depth : 9, accuracy_score : 0.9990754069964772,precision_score : 0.7804878048780488,recall_score : 0.6486486486486487
 Num : 5 , max_features : 3  , max_depth : 2, accuracy_score : 0.9991807403766253,precision_score : 0.8421052631578947,recall_score : 0.6486486486486487
 Num : 6 , max_features : 3  , max_depth : 5, accuracy_score : 0.9992509626300574,precision_score : 0.868421052631579,recall_score : 0.668918918918919
 Num : 7 , max_features : 3  , max_depth : 7, accuracy_score : 0.9994382219725431,pr

In [84]:
DT_result#.sort_values(by='recall_score', ascending = False)

,Num,max_features,max_depth,accuracy_score,precision_score,recall_score


SyntaxError: ignored

# 4, Random Forest Classification

# 5, Gradient Boosting Machine(GBM) Classification

# 6, Adaptive Boosting Classifier

# 7, eXtra Cradient Boost(XGBoost) Classification

# 8, LightGBM Classification

# 9, K-Nearest Neighbors Classification

In [ ]:
def lightGBM(model_name, X, y, n_est, md=None, lr=0.5,  visualFeature = False):
  # split  
  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=2045, stratify = y)

  # SMOTE
  os = SMOTE(random_state = 2045)
  X_train_os, y_train_os = os.fit_sample(X_train, y_train)

  # model
  model_lgbm = LGBMClassifier(n_estimators = n_est, max_depth =md, learning_rate = lr, reg_lambda= 0.2, n_jobs= -1, random_state=2045)
  model_lgbm.fit(X_train_os, y_train_os)

  # evaluate
  score_ac = accuracy_score(y_test, model_lgbm.predict(X_test))
  score_rc = recall_score(y_test, model_lgbm.predict(X_test), pos_label =1 )
  score_pr = precision_score(y_test, model_rf.predict(X_test), pos_label =1 )

  if visualFeature:
    plt.figure(figsize = (9, 6))
    sns.barplot(model_lgbm.feature_importances_, X.columns)
    plt.title('Feature Importance')
    plt.show()

  return {'model_id':model_name, 'n_estimators':n_est, 'max_depth':md, 'learning_rage':lr, 'Accuracy_score':score_ac, 'precision_score':score_pr,'Recall_score':score_rc}

In [ ]:
X = DF[['V10','V14','V17']]
y = DF['Class']
DF_lgbm_smote = pd.DataFrame(data=None, columns = ['model_id','n_estimators','max_features','max_depth','Accuracy_score','precision_score', 'Recall_score'])

In [ ]:
X_train_os.shape, y_train_os.shape

NameError: ignored

In [ ]:
%%time
a = [50, 100, 300]
index = 1
for n in a:
  model_name = 'RF_3-'+str(index)
  index += 1
  res = randomForest(model_name, X, y, n_est=n)
  print(res)
  DF_lgbm_smote = DF_lgbm_smote.append(res, ignore_index=True)
  DF_lgbm_smote

{'model_id': 'RF_3-1', 'n_estimators': 50, 'max_features': 'auto', 'max_depth': None, 'Accuracy_score': 0.9995903701883126, 'Recall_score': 0.8445945945945946}
{'model_id': 'RF_3-2', 'n_estimators': 100, 'max_features': 'auto', 'max_depth': None, 'Accuracy_score': 0.9995318516437859, 'Recall_score': 0.8445945945945946}
{'model_id': 'RF_3-3', 'n_estimators': 300, 'max_features': 'auto', 'max_depth': None, 'Accuracy_score': 0.9995318516437859, 'Recall_score': 0.8445945945945946}
CPU times: user 5min 10s, sys: 651 ms, total: 5min 10s
Wall time: 2min 38s


In [ ]:
%%time
a = [2,3,5,9]
index += 1
for n in a:
  model_name = 'RF_3-'+str(index)
  index += 1
  res = randomForest(model_name, X, y, n_est=50 , md=n)
  print(res)
  DF_lgbm_smote = DF_lgbm_smote.append(res, ignore_index=True)

{'model_id': 'RF_3-5', 'n_estimators': 50, 'max_features': 'auto', 'max_depth': 2, 'Accuracy_score': 0.9992158515033414, 'Recall_score': 0.668918918918919}
{'model_id': 'RF_3-6', 'n_estimators': 50, 'max_features': 'auto', 'max_depth': 3, 'Accuracy_score': 0.9993679997191109, 'Recall_score': 0.7567567567567568}
{'model_id': 'RF_3-7', 'n_estimators': 50, 'max_features': 'auto', 'max_depth': 5, 'Accuracy_score': 0.9994382219725431, 'Recall_score': 0.8243243243243243}
{'model_id': 'RF_3-8', 'n_estimators': 50, 'max_features': 'auto', 'max_depth': 9, 'Accuracy_score': 0.9995552590615966, 'Recall_score': 0.8445945945945946}
CPU times: user 1min 7s, sys: 294 ms, total: 1min 7s
Wall time: 35.7 s


In [ ]:
%%time
a = [1,2,3]
for n in a:
  model_name = 'RF_3-'+str(index)
  index += 1
  res = randomForest(model_name, X, y, n_est=50, md=9 , mf=n)
  print(res)
  DF_lgbm_smote = DF_lgbm_smote.append(res, ignore_index=True)

{'model_id': 'RF_3-9', 'n_estimators': 50, 'max_features': 1, 'max_depth': 9, 'Accuracy_score': 0.9995552590615966, 'Recall_score': 0.8445945945945946}
{'model_id': 'RF_3-10', 'n_estimators': 50, 'max_features': 2, 'max_depth': 9, 'Accuracy_score': 0.9995318516437859, 'Recall_score': 0.831081081081081}
{'model_id': 'RF_3-11', 'n_estimators': 50, 'max_features': 3, 'max_depth': 9, 'Accuracy_score': 0.9994967405170698, 'Recall_score': 0.8175675675675675}
CPU times: user 2min 28s, sys: 295 ms, total: 2min 28s
Wall time: 1min 16s


In [ ]:
DF_lgbm_smote.sort_values(by='Recall_score', ascending=False)

,model_id,n_estimators,max_features,max_depth,Accuracy_score,precision_score,Recall_score
0,RF_3-1,50,auto,None,0.999590,NaN,0.844595
1,RF_3-2,100,auto,None,0.999532,NaN,0.844595
2,RF_3-3,300,auto,None,0.999532,NaN,0.844595
6,RF_3-8,50,auto,9,0.999555,NaN,0.844595
7,RF_3-9,50,1,9,0.999555,NaN,0.844595
8,RF_3-10,50,2,9,0.999532,NaN,0.831081
5,RF_3-7,50,auto,5,0.999438,NaN,0.824324
9,RF_3-11,50,3,9,0.999497,NaN,0.817568
4,RF_3-6,50,auto,3,0.999368,NaN,0.756757
3,RF_3-5,50,auto,2,0.999216,NaN,0.668919
